<a href="https://www.kaggle.com/code/nadaahassan/face-analysis?scriptVersionId=142758491" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D
import os
Id=0
for dirname, _, filenames in os.walk('/kaggle/input/utkface-new/UTKFace'):
    for filename in filenames:
        
        if(Id==5):
            break 
        Id=Id+1
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:

def load(folder_path):
    
    image_paths = []
    age_labels = []
    gender_labels = []

    for filename in os.listdir(data_dir):
        image_path = os.path.join(data_dir, filename)
        temp = filename.split('_')
        age = int(temp[0])
        gender = int(temp[1])
        image_paths.append(image_path)
        age_labels.append(age)
        gender_labels.append(gender)
        df = pd.DataFrame()
    df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels
    return df 

In [ ]:
data_dir = '/kaggle/input/utkface-new/UTKFace/'
df = load(data_dir)
df.head()

In [ ]:
import os
import cv2
import numpy as np

def prepare(folder_path, target_size=(128, 128)):
    images = []
    image_names = []  
    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, target_size)
            images.append(img)
            image_name_without_extension = os.path.splitext(img_name)[0]
            image_names.append(image_name_without_extension)
            
    images = np.array(images)
    return images

data_dir = '/kaggle/input/utkface-new/UTKFace/'
images = prepare(data_dir)


In [ ]:
for i in range(1,5):
    plt.subplot(1, 5, i+1)
    plt.imshow(images[i])
    plt.title(df.age[i]) 
    plt.axis('off')
plt.show()

In [ ]:
sns.distplot(df['age'])

In [ ]:
sns.histplot(df['gender'])


In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
sns.boxplot(data=df, y='age')
plt.title('Box Plot of Age')
plt.show()

In [ ]:
images.shape

In [ ]:
X = images/255.0

In [ ]:
y_gen = np.array(df['gender'])
y_age = np.array(df['age'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    
    restore_best_weights=True 
)

In [ ]:
input_shape = X.shape[1:]

input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
dense_1 = Dense(256, activation='relu') (x)
dense_2 = Dense(256, activation='relu') (x)

dropout_1 = Dropout(0.3) (dense_1)
dropout_2 = Dropout(0.3) (dense_2)

age_out = Dense(1,  activation='relu',name='age_output')(x)
gender_out = Dense(1, activation='sigmoid', name='gender_output')(x)

model = Model(inputs=input_layer, outputs=[age_out, gender_out])

model.compile(optimizer='adam',
              loss={'age_output': 'mean_squared_error', 'gender_output': 'binary_crossentropy'},
              metrics={'age_output': 'mae', 'gender_output': 'accuracy'})
history = model.fit(
    X, {'age_output': y_age, 'gender_output': y_gen},
    epochs=30,              
    batch_size=40,
    validation_split=0.2,
    callbacks=[early_stopping] 
)


In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()




In [ ]:
gender_dict = {0:'Male', 1:'Female'}

In [ ]:
image_index = 60


print("Original Gender:", gender_dict[y_gen[image_index]], "Original Age:", y_age[image_index])
original_image = images[image_index]  
plt.imshow(original_image)
plt.show()

input_image = cv2.resize(original_image, (128, 128))
input_image = input_image.reshape(1, 128, 128, 3)

input_image = input_image / 255.0

pred = model.predict(input_image)

pred_gender = gender_dict[round(pred[1][0][0])]
pred_age = round(pred[0][0][0])  


print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)
